In [5]:
#pip install ipython-sql

In [6]:
#loading the extension
%load_ext sql

#connectin to sqlite database
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
#importing pandas module
import pandas as pd

In [8]:
#loading grand electors
with open('GrandElectors_by_state.csv', 'r') as f:
  electors = pd.read_csv(f, encoding='utf-8', index_col=0)

#cheking if a table with the same name grand if yes drop it and persist a new one we just loaded
%sql DROP TABLE if EXISTS electors
%sql PERSIST electors

FileNotFoundError: [Errno 2] No such file or directory: 'GrandElectors_by_state.csv'

In [ ]:
#loading populations by state
with open('Population_by_state.csv', 'r') as f:
  statePopulation = pd.read_csv(f, encoding='utf-8', index_col=0)

#cheking if a table with the same name grand if yes drop it and persist a new one we just loaded
%sql DROP TABLE if EXISTS statePopulation
%sql PERSIST statePopulation

In [ ]:
#changing the grand electors on column state to uppercase to allow joining

%%sql 
UPDATE electors
SET `State` = UPPER(`State`);

In [ ]:
#Forming a new table called state_data to contain all our data

%%sql

CREATE TABLE state_data as 
SELECT electors.State, Population, electors.GrandElectors From statePopulation INNER JOIN electors
ON statePopulation.State = electors.State;


In [ ]:
#viewing the first 5 data of our state_data table to verify the inner join was a success
%%sql

SELECT * FROM state_data
limit 5;

In [ ]:
#updating the state DISTRICT OF COLUMBIA to DC
%%sql

UPDATE state_data
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA";

In [ ]:
#To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.

#Creating a column nammed Ratio 
%%sql
ALTER TABLE `state_data` ADD COLUMN Ratio DECIMAL(38,38);

In [ ]:
#computing the ratio of grandelectors to population then we update this ratio to the Ratio column
%%sql
UPDATE `state_data`
SET `Ratio` = CAST(`GrandElectors` AS FLOAT) / CAST(`Population` AS FLOAT) ;


In [ ]:
#To order the states by decreasing ratio of Grand Electors per capita.

#To list the states in decreasing ratio we will be forced to create a new table called decreasing_per_capita

%%sql

CREATE TABLE decreasing_per_capita as
SELECT * FROM `state_data`
ORDER BY `Ratio` DESC;

In [ ]:
#a view of the table with decreasing Grand electors per capita
%%sql
SELECT * FROM decreasing_per_capita;

In [ ]:
#To compute the running total of Grand Electors in that sorted list

#we now create a table with a column for our running total. We call this table with_running_total
%%sql
CREATE TABLE `with_running_total` as
SELECT *, (SELECT SUM(r.GrandElectors) FROM decreasing_per_capita as r WHERE r.Ratio >= t.Ratio) as `Running Total`
FROM decreasing_per_capita as t;

In [ ]:
#We now query the table to view what we have done on th running total
%%sql
SELECT * FROM `with_running_total`;

In [ ]:
#Independently, to compute the half of the total of Grand Electors overall (in the whole country):
#This is the threshold we need to reach for winning the presidential election.

#From our running total we see that it sums to 538 (grandelectors) this means half way will have attained our desirable
#threshold

%%sql
SELECT (SUM(`GrandElectors`)/2) as `Threshold Target` FROM `with_running_total`;

In [ ]:
#To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. 

#to perform this operation we make sure we have all the state with running total below 269 the query the minimum value
#From the list that had running total above our threshold
%%sql
SELECT * FROM `with_running_total`
where `Running Total` < 269 or `Running Total` in (select MIN(`Running Total`) FROM `with_running_total` where `Running Total` > 269 )


In [ ]:
#number of state to target
%%sql
SELECT count(state) as `States To Target` FROM `with_running_total`
where `Running Total` < 269 or `Running Total` in (select MIN(`Running Total`) FROM `with_running_total` where `Running Total` > 269 )



<h1 style='color':red>CONCLUSIONS</h1>
<p><b>We end up with 40 states to focus on from our analysis. This is based on assumption that the ratio of grand electors
to population will produce the desirable result. It is evident that this proportioning was not a real reflection to 
acquaring the optimal strategy since the rationing of grand electors to population is not linear. A good example is 
focusing on califonia alone will free us from 14 states. We hereby conclude that this analysis doesn't give the desirable
optimal stratey since focusing on 40 states (70.43%) out of the possible 51 is not cost effective. <b></p>